## Documentation

- API Guide can be found in this link https://ib-insync.readthedocs.io/api.html
- Options Example (SPX) with ib_insync https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [63]:
from ib_insync import *
import pandas as pd
util.startLoop()
from datetime import date
from datetime import datetime

def verify_daycount(dates):
    ds = []
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    for adate in dates:
        date_obj = datetime.strptime(adate, "%Y%m%d")
        days_diff = (date_obj - today).days
        if (36 < days_diff < 55) or (90 < days_diff < 150):
            ds.append(adate)
    return ds
    

ib = IB()
ib.connect('127.0.0.1', 7496, clientId = 7)

tick = 'AAPL'
print( " >>> DEFINING STOCK ")
stk = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(stk)
ib.reqMarketDataType(4) #wait a bit
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
ds_chains = pd.DataFrame(chains)
expirations = sorted(ds_chains['expirations'].iloc[0])
expirations = verify_daycount(expirations) #convert expiration day to number of days left to expire
strikes = ds_chains['strikes'].iloc[0]
contractID = ds_chains['underlyingConId'].iloc[0]
rights = ['P', 'C'] #easier for algirthm built # P = Put ; C = Call

#organize all Option Contracts into data set
print( " >>> DEFINING OPTIONS ")
contracts = [Option(tick, ex, s, r, 'SMART') for r in rights for ex in expirations for s in strikes]
print( " >>> QUALIFYING OPTIONS ")
qualifiedContracts = ib.qualifyContracts(*contracts)
print( " >>> RETRIEVING DETAILS ")
qualifiedContractDetails = ib.reqTickers(*qualifiedContracts)
ds = pd.DataFrame()
print ( " >>> GETTING GREEKS AND OPTIONS INFO")
for qcd in qualifiedContractDetails:
    try:
        details = qcd.contract
        exp, strike, right = details.lastTradeDateOrContractMonth, details.strike, details.right
        greek = qcd.modelGreeks
        delta, theta, gamma, vega, pvDiv, optPrice, impliedVol = greek.delta, greek.theta, greek.gamma, greek.vega, greek.pvDividend, greek.optPrice, greek.impliedVol
        row = {
            'Symbol': tick, 'Expiration': exp , 'Strike': strike, 'Right': right,
            'Delta': delta, 'Theta': theta, 'Vega': vega, 'Gamma': gamma,
            'impVol': impliedVol, 'optPrice': optPrice, 'Ask': qcd.ask, 'Bid': qcd.bid, 'Last': qcd.last}
        ds = pd.concat([ds, pd.DataFrame([row])], axis = 0, ignore_index = True)
    except:
        print ( " >>> NOGOOD CONTRACT")

#next: call all options during market time to get greek values
def days_diff(adate):
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    date_obj = datetime.strptime(adate, "%Y%m%d")
    return (date_obj - today).days

ds['daysdiff'] = list(map(days_diff, ds['Expiration'])) 
ds['Delta'] = abs(ds['Delta'])
ds['Theta'] = abs(ds['Theta'])
print(ds)

# ds_p1 = ds[(0.1 <= ds['Delta'] <= 0.2) & (ds['Right'] == 'P') & (days_diff(ds['Expiration'], 90, 150))]
ds_p1 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_p1['REGEL'] = ['P1'] * len(ds_p1)
ds_p2 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 36) & (ds['daysdiff'] <= 55) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_p2['REGEL'] = ['P2'] * len(ds_p2)
ds_c3 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 36) & (ds['daysdiff'] <= 55) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_c3['REGEL'] = ['C3'] * len(ds_c3)
ds_c4 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_c4['REGEL'] = ['C4'] * len(ds_c4)

main = pd.concat([ds_p1, ds_p2, ds_c3, ds_c4], ignore_index=True)
def p_marg(p1, p2):
    if (p1 == p2 - 10) | (p1 == p2 - 20):
        return True
    return False

def c_marg(c3, c4):
    if (c4 == c3 + 10) | (c4 == c3 + 20):
        return True
    return False

def p_theta(p1, p2):
    if (p2 > p1 * 1.5):
        return True
    return False

def c_theta(c3, c4):
    if (c3 > c4 * 1.5):
        return True
    return False

def bundle(ds):
    p1 = ds[ds['REGEL'] == 'P1']
    p2 = ds[ds['REGEL'] == 'P2']
    c3 = ds[ds['REGEL'] == 'C3']
    c4 = ds[ds['REGEL'] == 'C4']
    ps = []
    cs = []
    i = 0

    #convert two loops to function prperly (rearrange functions in sheet)
    for i in range(len(p1)):
        j = 0
        for j in range(len(p2)):
            # code begin
            if p_marg(p1.iloc[i]['Strike'], p2.iloc[j]['Strike']) & p_theta(p1.iloc[i]['Theta'], p2.iloc[j]['Theta']):
                ps.append((i, j))
            # code end
            j += 1
        i += 1

    for i in range(len(c3)):
        j = 0
        for j in range(len(c4)):
            # code begin
            if c_marg(c3.iloc[i]['Strike'], c4.iloc[j]['Strike']) & c_theta(c3.iloc[i]['Theta'], c4.iloc[j]['Theta']):
                cs.append((i, j))
            # code end
            j += 1
        i += 1
    

    legs_indx = [(p1, p2, c3, c4) for (p1, p2) in ps for (c3, c4) in cs]

    dss = []
    for (a, b, c, d) in legs_indx:
        ds = pd.concat([p1.iloc[a].to_frame().T, p2.iloc[b].to_frame().T, c3.iloc[c].to_frame().T, c4.iloc[d].to_frame().T], ignore_index=True)
        # df = pd.DataFrame.from_records(ds)
        dss.append(ds)
    
    return dss


bundles = bundle(main)
pd.concat(bundles).to_csv(tick+'_options.csv')











ib.disconnect()


Error 200, reqId 5: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=41.25, right='P', exchange='SMART')
Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=42.5, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has b

 >>> DEFINING STOCK 
 >>> DEFINING OPTIONS 
 >>> QUALIFYING OPTIONS 


Error 200, reqId 39: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=97.5, right='P', exchange='SMART')
Error 200, reqId 41: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=102.5, right='P', exchange='SMART')
Error 200, reqId 43: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=107.5, right='P', exchange='SMART')
Error 200, reqId 45: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=112.5, right='P', exchange='SMART')
Error 200, reqId 47: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=117.5, right='P', exchange='SMART')
Error 200, reqId 48: No security definiti

 >>> RETRIEVING DETAILS 
 >>> GETTING GREEKS AND OPTIONS INFO
    Symbol Expiration  Strike Right     Delta     Theta      Vega     Gamma  \
0     AAPL   20220826    70.0     P  0.001004  0.001831  0.001785  0.000083   
1     AAPL   20220826    75.0     P  0.001410  0.002366  0.002178  0.000120   
2     AAPL   20220826    80.0     P  0.001970  0.003002  0.003394  0.000173   
3     AAPL   20220826    85.0     P  0.002555  0.003543  0.003499  0.000236   
4     AAPL   20220826    90.0     P  0.003203  0.004039  0.006126  0.000312   
..     ...        ...     ...   ...       ...       ...       ...       ...   
381   AAPL   20221216   270.0     C  0.003398  0.001089  0.011975  0.000332   
382   AAPL   20221216   275.0     C  0.002775  0.000910  0.008531  0.000274   
383   AAPL   20221216   280.0     C  0.002006  0.000681  0.008649  0.000205   
384   AAPL   20221216   285.0     C  0.001593  0.000550  0.005196  0.000165   
385   AAPL   20221216   290.0     C  0.001255  0.000444  0.004775  0.

<ipython-input-63-fb70554a2d58>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_p1['REGEL'] = ['P1'] * len(ds_p1)
<ipython-input-63-fb70554a2d58>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_p2['REGEL'] = ['P2'] * len(ds_p2)
<ipython-input-63-fb70554a2d58>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [18]:
print(bundles)
print(main)
ib.disconnect()

[  Symbol Expiration Strike Right     Delta     Theta      Vega     Gamma  \
0   AAPL   20221021  120.0     P  0.111552  0.030164  0.142668  0.006142   
1   AAPL   20220826  140.0     P   0.25607  0.075312  0.163071  0.017717   
2   AAPL   20220826  160.0     C  0.276375   0.06523  0.158862  0.023519   
3   AAPL   20221021  180.0     C  0.110478  0.021486  0.151456  0.009299   

     impVol  optPrice   Ask   Bid  Last daysdiff REGEL  
0  0.407256   1.84287  1.88  1.86  1.87       94    P1  
1  0.377206  2.998884   NaN   NaN   NaN       38    P2  
2  0.293912  2.324134  2.38  2.32   2.3       38    C3  
3  0.266304  1.031228  1.03  1.01  1.01       94    C4  ,   Symbol Expiration Strike Right     Delta     Theta      Vega     Gamma  \
0   AAPL   20221021  120.0     P  0.111552  0.030164  0.142668  0.006142   
1   AAPL   20220826  140.0     P   0.25607  0.075312  0.163071  0.017717   
2   AAPL   20220826  160.0     C  0.276375   0.06523  0.158862  0.023519   
3   AAPL   20221118  180.0  

In [81]:
import time

# Create the main line at each bundle sheet
ib.connect('127.0.0.1', 7496, clientId = 13)
contract = Stock('AAPL','SMART','USD')
# ib = IB().connect('127.0.0.1', 4001 , clientId=100) #connect
m_data = ib.reqMktData(contract)
while m_data.last != m_data.last: ib.sleep(0.01) #Wait until data is in. 
ib.cancelMktData(contract)
last_price = m_data.last

stock = Stock(tick, 'SMART', 'USD') #, primaryExchange='NASDAQ'
bars = ib.reqHistoricalData(stock, endDateTime = '', durationStr = '26 W',barSizeSetting = '1 hour', whatToShow = 'MIDPOINT', useRTH = True)
lows, highs = [], []

for bar in bars:
    lows.append(bar.low)
    highs.append(bar.high)

bars = ib.reqHistoricalData(stock, endDateTime = '', durationStr = '2 D',barSizeSetting = '1 day', whatToShow = 'MIDPOINT', useRTH = True)
change = (bars[0].close * 100) / last_price


ds__ = pd.DataFrame({'lows': lows,'highs': highs})
mini = ds__['lows'].min()
maxi = ds__['highs'].max()
ib.disconnect()

d = " - "
mids = { #main info dataset
'Symbol': tick, 
'Last': last_price,
'Change%': round(change, 2),
'Low-26W': round(mini, 2),
'High-26W': round(maxi, 2),
"IV%": d,
"H-IV-30%": d,
"H-IV-Mean%": 25200,
"RSI-14D": d
}

mids = pd.DataFrame([mids])
print(mids)


  Symbol    Last  Change%  Low-26W  High-26W  IV% H-IV-30%  H-IV-Mean% RSI-14D
0   AAPL  152.64    98.91   129.07     179.6   -        -        25200      - 


In [49]:
ib.disconnect()

In [82]:
import openpyxl
import xlsxwriter
import numpy as np
# bundles.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

writer = pd.ExcelWriter('bundles.xlsx',engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Result')
writer.sheets['Bundles'] = worksheet

mids.to_excel(writer, sheet_name = 'Bundles', startrow = 0, startcol = 0)


bundle_count = 1
currow, curcol = 4, 0
for ds in bundles:
        symbol = str(list(ds[ds['REGEL'] == 'P1']['Symbol'])[0])
        worksheet.write_string(currow, 0, str(symbol) + ": Bundle " + str(bundle_count))
        currow += 1
        ds['Change'] = ((ds['Strike'] - last_price)/last_price) #convert to percentage
        ds['Change'] = ds['Change'].apply(lambda x: round(x, 2))
        ds['Mark'] = (ds['Ask'] + ds['Bid']) / 2
        ds[ds['REGEL'] == 'P2']['Mark'] = -1 * ds[ds['REGEL'] == 'P2']['Mark']
        ds[ds['REGEL'] == 'C3']['Mark'] = -1 * ds[ds['REGEL'] == 'C3']['Mark']
        ds['Mark'] = ds['Mark'].apply(lambda x: round(x, 2))
        ds = ds.rename({'impVol': 'IV-OP'}, axis = 1)
        ds['IV-OP'] = ds['IV-OP'].apply(lambda x: round(x, 2))
        # ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')
        ds['Expiration'] = pd.to_datetime(ds.Expiration, format='%Y%m%d')
        ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')

        ds = ds.rename({'daysdiff': 'Days'}, axis = 1) 
        ds['Delta'] = ds['Delta'].apply(lambda x: round(x, 2))
        ds['Theta'] = ds['Theta'].apply(lambda x: round(x, 2))
        ds['Vega'] = ds['Vega'].apply(lambda x: round(x, 2))


        rtd_p, rtd_c = list(ds[ds['REGEL'] == 'P2']['Theta'])[0] - list(ds[ds['REGEL'] == 'P1']['Theta'])[0], list(ds[ds['REGEL'] == 'C3']['Theta'])[0] - list(ds[ds['REGEL'] == 'C4']['Theta'])[0]
        ds['RTD'] = [rtd_p, rtd_p, rtd_c, rtd_c]
        ds['RTM'] = ds['RTD'] * 22
        ds['RTY'] = ds['RTM'] * 12
        mp, mc = list(ds[ds['REGEL'] == 'P2']['Strike'])[0] - list(ds[ds['REGEL'] == 'P1']['Strike'])[0], list(ds[ds['REGEL'] == 'C4']['Strike'])[0] - list(ds[ds['REGEL'] == 'C3']['Strike'])[0]
        ds['Margin'] = [mp, mp, mc, mc]
        ds['RTM'] = ds['RTM'].apply(lambda x: round(x, 2))
        ds[['REGEL', 'Expiration', 'Days', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'IV-OP', 'RTM']].to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += ds.shape[0] + 2


        mc = mc.astype(np.int64)
        mp = mp.astype(np.int64)
        
        if mc > mp:
                max_margin = mc
        else:
                max_margin = mp

        pnative = {
        'Type': "P-Native", 
        'Margin': mp,
        'Monthly%': (rtd_p * 22 * 100) / mp,
        'Yearly%': (rtd_p * 22 * 12 * 100) / mp,
        'Yearly$': rtd_p * 22 * 12,
        "Cost": list(ds[ds['REGEL'] == 'P1']['Mark'])[0] + list(ds[ds['REGEL'] == 'P2']['Mark'])[0]
        }

        cnative = {
        'Type': "C-Native", 
        'Margin': mc,
        'Monthly%': (rtd_c * 22 * 100) / mc,
        'Yearly%': (rtd_c * 22 * 12 * 100) / mc,
        'Yearly$': (rtd_c * 22 * 12),
        "Cost": list(ds[ds['REGEL'] == 'C3']['Mark'])[0] + list(ds[ds['REGEL'] == 'C4']['Mark'])[0]
        }

        posnative = {
        'Type': "POS-Native", 
        'Margin': max_margin,
        'Monthly%': (rtd_c * 22 + rtd_p * 22) / max_margin,
        'Yearly%': ((rtd_c * 22 * 12) + (rtd_p * 22 * 12)) / max_margin,
        'Yearly$': (pnative['Yearly$'] + cnative['Yearly$']),
        "Cost": ds['Mark'].sum()
        }

        ds_ = pd.DataFrame([pnative, cnative, posnative])
        ds_.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 7 # 4 for the native ds_, 2 for spacing
        bundle_count += 1

writer.save() #save the excelt sheet

<ipython-input-82-e0ad171169e4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds[ds['REGEL'] == 'P2']['Mark'] = -1 * ds[ds['REGEL'] == 'P2']['Mark']
<ipython-input-82-e0ad171169e4>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds[ds['REGEL'] == 'C3']['Mark'] = -1 * ds[ds['REGEL'] == 'C3']['Mark']
<ipython-input-82-e0ad171169e4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [ ]:
[Ticker(
    contract=Option(symbol='SPY', lastTradeDateOrContractMonth='20180420', strike=270, right='C', exchange='SMART'), 
    time=datetime.datetime(2018, 3, 1, 16, 1, 17, 467516, tzinfo=datetime.timezone.utc), 
    bid=6.83, bidSize=1156, ask=6.88, askSize=11, last=6.88, lastSize=1, volume=2264, high=7.67, low=6.17, close=6.95, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], 
    bidGreeks=OptionComputation(impliedVol=0.160298280414867, delta=None, optPrice=6.829999923706055, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    askGreeks=OptionComputation(impliedVol=0.16155485178197637, delta=None, optPrice=6.880000114440918, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=6.880000114440918, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    modelGreeks=OptionComputation(impliedVol=0.15770592183303841, delta=0.5344203452034115, optPrice=6.665772690114447, pvDividend=1.0992683616305827, gamma=0.026063908021373355, vega=0.3941542814583521, theta=-0.07095144483744885, undPrice=271.32000732421875)
)]
